In [1]:
# Самостоятельное задание  №3. Фильмы и актеры. С использованием временных таблиц и SQL

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession,DataFrame
from pyspark.sql.functions import col, round, to_date, year, month, sum, avg

conf = (
     SparkConf()
    .setAppName('Simple_Spark')
    .setMaster('local[*]')
)


# Spark сессия
spark = (
    SparkSession
    .builder
    .config(conf=conf)
    # .enableHiveSupport()
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
!hdfs dfs -ls -h ./data_sets

Found 8 items
-rw-r-----   3 akbogdanov hdfs        869 2024-09-06 12:21 data_sets/actors.csv
-rw-r-----   3 akbogdanov hdfs      7.5 M 2024-04-15 16:09 data_sets/airport-codes.csv
-rw-r-----   3 akbogdanov hdfs        314 2024-09-06 11:11 data_sets/authors.csv
-rw-r-----   3 akbogdanov hdfs        824 2024-09-06 11:11 data_sets/books.csv
-rw-r-----   3 akbogdanov hdfs        283 2024-09-06 12:20 data_sets/movie_actors.csv
-rw-r-----   3 akbogdanov hdfs        854 2024-09-06 12:21 data_sets/movies.csv
-rw-r--r--   3 akbogdanov hdfs      2.1 K 2023-09-04 22:22 data_sets/speech_ivr.csv
-rw-r-----   3 akbogdanov hdfs     75.1 K 2024-09-05 16:21 data_sets/weather_data.csv


### Подготовка таблицы movies

In [21]:
movies_data = spark.read.csv("./data_sets/movies.csv", header=True)
movies_df = (movies_data
    .withColumn("budget", col("budget").cast("Double"))
    .withColumn("release_date", to_date(col("release_date")))
            )
movies_df.show()
movies_df.createOrReplaceTempView("movies")

+--------+--------+------+------------+-------------+
|movie_id|   title| genre|release_date|       budget|
+--------+--------+------+------------+-------------+
|       1| Movie_1|Horror|  2000-12-31|8.660058311E7|
|       2| Movie_2|Comedy|  2001-12-31|1.274740083E7|
|       3| Movie_3|Action|  2002-12-31| 1.80157747E7|
|       4| Movie_4| Drama|  2003-12-31|4.817612061E7|
|       5| Movie_5| Drama|  2004-12-31| 7.40501611E7|
|       6| Movie_6|Action|  2005-12-31|1.476121831E7|
|       7| Movie_7| Drama|  2006-12-31|4.456703643E7|
|       8| Movie_8| Drama|  2007-12-31|4.880227617E7|
|       9| Movie_9|Action|  2008-12-31|2.201627853E7|
|      10|Movie_10|Action|  2009-12-31|1.244027929E7|
|      11|Movie_11|Comedy|  2010-12-31|8.380567138E7|
|      12|Movie_12|Comedy|  2011-12-31|5.074409933E7|
|      13|Movie_13|Action|  2012-12-31|   2423742.36|
|      14|Movie_14|Sci-Fi|  2013-12-31|8.049514883E7|
|      15|Movie_15| Drama|  2014-12-31|9.809858674E7|
|      16|Movie_16|Comedy|  

In [22]:
movies_df.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- budget: double (nullable = true)



### Подготовка таблицы actors

In [16]:
actors_data = spark.read.csv("./data_sets/actors.csv", header=True)
actors_df = actors_data.withColumn("birth_date", to_date(col("birth_date")))
actors_df.show()
actors_df.createOrReplaceTempView("actors")

+--------+--------+----------+---------+
|actor_id|    name|birth_date|  country|
+--------+--------+----------+---------+
|       1| Actor_1|1960-12-31|   Canada|
|       2| Actor_2|1962-12-31|       UK|
|       3| Actor_3|1964-12-31|       UK|
|       4| Actor_4|1966-12-31|       UK|
|       5| Actor_5|1968-12-31|    India|
|       6| Actor_6|1970-12-31|      USA|
|       7| Actor_7|1972-12-31|    India|
|       8| Actor_8|1974-12-31|Australia|
|       9| Actor_9|1976-12-31|      USA|
|      10|Actor_10|1978-12-31|Australia|
|      11|Actor_11|1980-12-31|      USA|
|      12|Actor_12|1982-12-31|    India|
|      13|Actor_13|1984-12-31|       UK|
|      14|Actor_14|1986-12-31|   Canada|
|      15|Actor_15|1988-12-31|       UK|
|      16|Actor_16|1990-12-31|    India|
|      17|Actor_17|1992-12-31|      USA|
|      18|Actor_18|1994-12-31|       UK|
|      19|Actor_19|1996-12-31|    India|
|      20|Actor_20|1998-12-31|Australia|
+--------+--------+----------+---------+
only showing top

In [18]:
actors_df.printSchema()

root
 |-- actor_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- country: string (nullable = true)



### Подготовка таблицы movie_actors

In [23]:
movie_actors_df = spark.read.csv("./data_sets/movie_actors.csv", header=True)
movie_actors_df.show()
movie_actors_df.createOrReplaceTempView("movie_actors")

+--------+--------+
|movie_id|actor_id|
+--------+--------+
|       1|      25|
|      16|       5|
|       6|      16|
|      16|      11|
|      14|      21|
|       3|       6|
|      15|       9|
|       3|      13|
|       2|      24|
|       1|       8|
|       9|      14|
|       9|      24|
|       7|       1|
|       3|      17|
|      18|      24|
|      11|       5|
|       7|      25|
|       9|       2|
|       1|      25|
|      14|      28|
+--------+--------+
only showing top 20 rows



In [24]:
movie_actors_df.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- actor_id: string (nullable = true)



In [41]:
# Найдите топ-5 жанров по количеству фильмов. movies actors movie_actors

spark.sql("""
SELECT m.genre
        ,COUNT(DISTINCT(title)) count_title
FROM movies m
JOIN movie_actors ma ON ma.movie_id = m.movie_id
JOIN actors a ON a.actor_id = ma.actor_id
GROUP BY m.genre
ORDER BY count_title DESC
LIMIT 5

""").show()



+------+-----------+
| genre|count_title|
+------+-----------+
|Action|          6|
| Drama|          4|
|Comedy|          4|
|Horror|          2|
|Sci-Fi|          2|
+------+-----------+



In [60]:
# Найдите актера с наибольшим количеством фильмов
spark.sql("""
SELECT name
      ,COUNT(m.movie_id) count_films
FROM movies m
JOIN movie_actors ma ON ma.movie_id = m.movie_id
JOIN actors a ON a.actor_id = ma.actor_id
GROUP BY a.name
ORDER BY count_films DESC
LIMIT 1

""").show()


+--------+-----------+
|    name|count_films|
+--------+-----------+
|Actor_17|          5|
+--------+-----------+



In [66]:
# Подсчитайте средний бюджет фильмов по жанрам.


spark.sql("""
SELECT m.genre
        ,AVG(DISTINCT(m.budget)) avg_budget
FROM movies m
JOIN movie_actors ma ON ma.movie_id = m.movie_id
JOIN actors a ON a.actor_id = ma.actor_id
GROUP BY m.genre
""").show()


+------+--------------------+
| genre|          avg_budget|
+------+--------------------+
| Drama|6.0427218525000006E7|
|Horror|      8.7281876775E7|
|Comedy|     5.20709662225E7|
|Action|2.7492742561666667E7|
|Sci-Fi|       7.809715175E7|
+------+--------------------+



In [76]:
# Найдите фильмы, в которых снялось более одного актера из одной страны.

spark.sql("""
SELECT m.title, a.country, COUNT(*) as num_actors
FROM movies m
JOIN movie_actors ma ON ma.movie_id = m.movie_id
JOIN actors a ON a.actor_id = ma.actor_id
GROUP BY m.title, a.country
HAVING num_actors > 1
ORDER BY num_actors DESC

""").show()

+--------+---------+----------+
|   title|  country|num_actors|
+--------+---------+----------+
| Movie_1|    India|         3|
| Movie_7|    India|         2|
| Movie_3|      USA|         2|
|Movie_10|       UK|         2|
|Movie_15|    India|         2|
|Movie_18|Australia|         2|
| Movie_2|      USA|         2|
| Movie_7|      USA|         2|
|Movie_10|      USA|         2|
+--------+---------+----------+



In [30]:

spark.sql("""
SELECT *
FROM movies m
JOIN movie_actors ma ON ma.movie_id = m.movie_id
JOIN actors a ON a.actor_id = ma.actor_id

""").show()

+--------+-------+------+------------+-------------+--------+--------+--------+--------+----------+---------+
|movie_id|  title| genre|release_date|       budget|movie_id|actor_id|actor_id|    name|birth_date|  country|
+--------+-------+------+------------+-------------+--------+--------+--------+--------+----------+---------+
|       1|Movie_1|Horror|  2000-12-31|8.660058311E7|       1|      13|      13|Actor_13|1984-12-31|       UK|
|       1|Movie_1|Horror|  2000-12-31|8.660058311E7|       1|      16|      16|Actor_16|1990-12-31|    India|
|       1|Movie_1|Horror|  2000-12-31|8.660058311E7|       1|      25|      25|Actor_25|2008-12-31|    India|
|       1|Movie_1|Horror|  2000-12-31|8.660058311E7|       1|       8|       8| Actor_8|1974-12-31|Australia|
|       1|Movie_1|Horror|  2000-12-31|8.660058311E7|       1|      25|      25|Actor_25|2008-12-31|    India|
|       2|Movie_2|Comedy|  2001-12-31|1.274740083E7|       2|      17|      17|Actor_17|1992-12-31|      USA|
|       2|